In [2]:
import cv2
import numpy as np

print(cv2.__version__)

4.5.1


In [3]:
cap = cv2.VideoCapture(0)
whT = 320
confThreshold = 0.5
nmsThreshold = 0.3

classesFile = 'obj.names'
classNames = []

with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    
print(classNames)
print(len(classNames))

modelConfiguration= 'yolov3_ED.cfg'
modelWeights= 'yolov3_best.weights'

net = cv2.dnn.readNetFromDarknet(modelConfiguration,modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

def findObjects(outputs,img):
    hT, wT, cT, = img.shape
    bbox= []
    classIds = []
    confs = []
    
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId= np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT), int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
    #print(len(bbox))
    indices = cv2.dnn.NMSBoxes(bbox,confs,confThreshold,nmsThreshold)
    print(indices)
    for i in indices:
        i = i[0]
        box = bbox[i]
        x,y,w,h = box[0],box[1],box[2],box[3]
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%', (x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
            

while True:
    success, img = cap.read()
    
    blob = cv2.dnn.blobFromImage(img,1/255,(whT,whT),[0,0,0],1,crop=False)
    net.setInput(blob)
    
    layerNames = net.getLayerNames()
    #print(layerNames)
    outputNames = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
    #print(outputNames) 
   # print(net.getUnconnectedOutLayers())

    outputs = net.forward(outputNames)
    print(outputs[0].shape)
    print(outputs[1].shape)
    print(outputs[2].shape)
    print(outputs[0][0])
    
    findObjects(outputs,img)
    
    cv2.imshow('Img',img)
    cv2.waitKey(1)
    

['Expiration date']
1
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.4171761e-02 2.8383356e-01 2.6051325e-01 5.3186882e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5203786e-02 2.7656907e-01 2.6704130e-01 6.3069015e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.000000e+00 6.485637e-02 2.762158e-01 2.647137e-01 5.251638e-06
 0.000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5876685e-02 2.8409532e-01 2.6004905e-01 5.5093815e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5594926e-02 2.7278370e-01 2.6572675e-01 6.2538593e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5652609e-02 2.7454954e-01 2.6551920e-01 5.1499842e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.4699575e-02 2.7391687e-01 2.6890025e-01 7.4935870e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5274194e-02 2.7651572e-01 2.6230961e-01 4.5595561e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.

(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5443873e-02 2.8966123e-01 2.4264355e-01 4.5188563e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5301113e-02 2.8656009e-01 2.5166255e-01 7.2104876e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.4261317e-02 2.9273731e-01 2.3242863e-01 3.0871206e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[2.9802323e-09 6.0188644e-02 3.0749494e-01 2.5668192e-01 9.8166420e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[5.6624412e-08 6.1449729e-02 3.1357643e-01 2.3032872e-01 7.7558198e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[1.3113022e-07 5.9202813e-02 2.8645411e-01 1.9343702e-01 2.6336595e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.1865091e-02 3.0453166e-01 2.3861000e-01 2.3613020e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[1.4901161e-07 5.2112810e-02 3.1695029e-01 1.6723333e-01 1.6251786e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[3.5762788e-08 5.47

(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 5.9580583e-02 3.0819285e-01 2.5836349e-01 4.3413552e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.2786914e-02 3.1812841e-01 2.7887550e-01 6.8873223e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 5.8022529e-02 2.9097959e-01 2.6713678e-01 2.2487975e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.3703820e-02 3.1408960e-01 2.3460349e-01 2.1842852e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5982446e-02 3.4022948e-01 2.2038479e-01 1.7086713e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.6491514e-02 3.1897905e-01 2.3131776e-01 2.7278122e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5814957e-02 3.1853095e-01 2.2511001e-01 2.5030745e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.000000e+00 6.323847e-02 3.015545e-01 2.490283e-01 5.179853e-06
 0.000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.7725696e

(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.7482695e-02 2.8253871e-01 2.3266318e-01 3.9670354e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.8179861e-02 2.8120428e-01 2.4687719e-01 4.5946990e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.4588100e-02 2.6843897e-01 2.5711894e-01 5.3347358e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.7731813e-02 2.7864307e-01 2.5016105e-01 4.4246340e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.6094682e-02 2.7153033e-01 2.6149204e-01 7.1534819e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5159500e-02 2.7526593e-01 2.5813356e-01 6.3730345e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.6696785e-02 2.8012553e-01 2.5154310e-01 5.2382329e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.5962613e-02 2.7305526e-01 2.6201516e-01 6.3394882e-06
 0.0000000e+00]
()
(300, 6)
(1200, 6)
(4800, 6)
[0.0000000e+00 6.44

KeyboardInterrupt: 